# Урок 2 - Исправление ошибок

В данном уроке рассказывается о способах обнаружения и исправления орфографических ошибок в словах.  
В основном для этой цели используют два алгоритма:
- Метод N-грамм (триграмы и биграмы)
- Растояние Левенштейна  
*Эти метода можно использовать как по отдельности так и вместе*

Мы разберем каждый из методов, а затем объеденим их

### Подготовка

Перед те как исправлять ошибки в словах их необходимо нормализовать. После этого множество разных слов станут одним и тем же словом, это позволит оставить действительно слова с ошибками а не одно и тоже слово, но в разных падежах

Для нормализации слова воспользуемся библиотекой pymorphy2

In [0]:
import pymorphy2
# обратите внимание на то, что анализатор потребляем около 20Мб оперативной памяти за 1 экземпляр класса
# поэтому не стоит держать много экземпляров в памяти и постоянно их создавать и уничтожать
# лучше создать один раз и использовать везде (петтерн Синглетон)
morph = pymorphy2.MorphAnalyzer()

In [0]:
word = morph.parse('студентами')[0]
word.normal_form

'студент'

С помощью этого можно приводить все слова в одну нормальную форму, что позволит существенно улучшить качество обработки текста

### Метод N-грамм

Данный метод является методом нечеткого сравнения строк и вводит функцию похожести одного слова на другое  

\begin{equation*}
P(s1, s2) = \frac{\sum_{i=1}^{min(N1, N2)}G(s1_{i}, s2_{i})}{N2} \quad (1) \\ 
G(s1, s2) = 
    \begin{cases}
    1 & \quad \text{если } s1 = s2 \\
    0 & \quad \text{если } s1 != s2
    \end{cases} \\
\text{где s1 - входное слово (N1 его длина), а s2  слово с которым происходит сравнение (N2 его длина),} \\
s1_{i} \text{ и } s2_{i} \text{ - подстрока строки s1 или s2 (по другому говоря это n-грамма),} \\
\text{длинна слова измеряется в n-граммах}
\end{equation*}

#### Описание алгоритма
Алгоритм основывается на принципе:
«Если слово А совпадает со словом Б с учетом нескольких ошибок, то с большой долей вероятности у них будет хотя бы одна общая подстрока длины N».
Эти подстроки длины N и называются N-граммами.
Во время индексации слово разбивается на такие N-граммы, а затем это слово попадает в списки для каждой из этих N-грамм. Во время поиска запрос также разбивается на N-граммы, и для каждой из них производится последовательный перебор списка слов, содержащих такую подстроку.

![img](https://habrastorage.org/storage/habraeffect/57/98/579859076e907968952471e047fd8827.png)

#### Алгоритм "Триграмный поиск"
##### Стадия 1 - Индексация
 - Каждое слово в словаре необходимо разбить на подстроки длиной 3 символа 
 - Сдвиг при разбиение на подстроки 1 сивол
 - Сохранить информацию о том какие триграммы соотвествуют какому слову
 
##### Стадия 2 - Сравнение
Шаги:
1. Разбить входное слово на триграммы
2. Найти все слова в которых встречаются триграммы входного слова
3. Подсчитать функцию похожести P **(1)** между входным словом и каждым словом, найденым на **шаге 2**
3. Выбрать слово с найбольшем значением функции P

Примечания:
- Если входное слово меньше 3 букв, то сразу же вернуть его
- Если имеются несколько слов с одинаковым значенем P, то не важно какое из них будет выведено
- Все слова в словаре должны быть нормализованы
- Входное слово перед разделением на триграммы должно быть нормализовано

**Дополнпительная модификация алгоритма**  
Вместо одного слова возвращать список слов отсортированых в порядке убывания значения фукции P

### Расстояние Левенштейна

Данный алгоритм в отличии от N-грамм не имеет стадии индексации и не использует словарь из-за чего он более медленный, но у данного алгоритма есть ряд преимуществ:
- Более точное сравнени (сравнение происходит посимвольно)
- Нет ограничения на минимальную длину слова
В основе алгоритма лежит следующая формула:

Пусть S1 и S2 — две строки (длиной M и N соответственно) над некоторым алфавитом, тогда редакционное расстояние (расстояние Левенштейна) d(S1, S2) можно подсчитать по следующей рекуррентной формуле

\begin{equation}
d(S_{1}, S_{2}) = D(M, N), \text{где} \quad \quad (2) \\
D(i, j) = \left\{\begin{array}{llcl}
0,&&&i = 0,\ j = 0\\
i,&&&j = 0,\ i > 0\\
j,&&&i = 0,\ j > 0\\
\min\{\\
&D(i, j - 1) + 1,\\
&D(i - 1, j) + 1,&&j > 0,\ i > 0\\
&D(i - 1, j - 1) + {\rm m}(S_1[i], S_2[j])\\
\}
\end{array}\right.
\end{equation}

где m(a,b) равна нулю, если a=b и единице в противном случае;  
min{a,b,c} возвращает наименьший из аргументов.

Здесь шаг по i символизирует удаление (D) из первой строки, по j — вставку (I) в первую строку, а шаг по обоим индексам символизирует замену символа (R) или отсутствие изменений (M).

Функция похожести двух строк для данного алгоритма:
\begin{equation}
P(s_1, s_2) = \frac{d(s_1, s_2)}{max(N_1, N_2)} \quad (3)
\end{equation}

##### Алгоритм
1. Нормализовать входное слово
2. Посчитать функцию **P (3)**

Примечание:
- Данный алгоритм не требует использования словаря, но для данного случая все таки необходимо сосдать словарь нормализованых слов
- s1 входное слово, s2 слово из словаря
- **При тестировании алгоритма не используйте большой словарь так как его обработка потребует значительного времени**

**Модификация алгоритма**  
Вместо метрике P возвращать список слов отсортированых в порядке убывания значения фукции P

## Задание
Варианты:
1. Реализовать триграммный поиск с модификацией
2. Реализовать Алгоритм "Растояние Левенштейна с модификацией"